In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Metabolic_Rate"
cohort = "GSE61225"

# Input paths
in_trait_dir = "../../input/GEO/Metabolic_Rate"
in_cohort_dir = "../../input/GEO/Metabolic_Rate/GSE61225"

# Output paths
out_data_file = "../../output/preprocess/Metabolic_Rate/GSE61225.csv"
out_gene_data_file = "../../output/preprocess/Metabolic_Rate/gene_data/GSE61225.csv"
out_clinical_data_file = "../../output/preprocess/Metabolic_Rate/clinical_data/GSE61225.csv"
json_path = "../../output/preprocess/Metabolic_Rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE61225_family.soft.gz', 'GSE61225_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Metabolic_Rate/GSE61225/GSE61225_family.soft.gz
Matrix file: ../../input/GEO/Metabolic_Rate/GSE61225/GSE61225_series_matrix.txt.gz
Background Information:
!Series_title	"Gene expression changes in blood RNA after swimming in a pool"
!Series_summary	"Trihalomethanes (THM) are a class of disinfection by-products in chlorinated waters linked to deleterious health effects in humans although biological mechanisms are unclear. We aimed to study short-term changes in blood gene expression of adult recreational swimmers associated with physical activity and THM exposure."
!Series_summary	"Adult volunteers (18-50 years, non-smokers, non-asthmatics) swam 40 minutes in an indoor chlorinated pool in Barcelona. Blood samples and THM measurements in exhaled breath were collected before and 5 min/1h after swimming, respectively. Physical activity intensity was calculated as metabolic equiva

### Step 2: Dataset Analysis and Clinical Feature Extraction

### Step 3: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Let's first examine the input data to determine what's available
import os
import pandas as pd
import numpy as np
import json
from typing import Optional, Callable, Dict, Any

# Let's check what files are available in the cohort directory
print(f"Files in {in_cohort_dir}:")
print(os.listdir(in_cohort_dir))

# Load and examine the clinical data
clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
if os.path.exists(clinical_data_path):
    clinical_data = pd.read_csv(clinical_data_path)
    print("\nClinical data shape:", clinical_data.shape)
    print("\nClinical data unique values:")
    for i, row in clinical_data.iterrows():
        unique_values = pd.Series(row.astype(str).unique())
        print(f"Row {i}: {unique_values.tolist()}")
else:
    print("Clinical data file not found.")

# Load and examine the background information
background_path = os.path.join(in_cohort_dir, "background.txt")
if os.path.exists(background_path):
    with open(background_path, 'r') as file:
        background_info = file.read()
    print("\nBackground information:")
    print(background_info)
else:
    print("Background information file not found.")

# Check if gene expression data is available
expression_path = os.path.join(in_cohort_dir, "expression_data.csv")
gene_file_exists = os.path.exists(expression_path)
print(f"\nGene expression data exists: {gene_file_exists}")

if gene_file_exists:
    # Peek at the expression data to ensure it's gene expression and not miRNA/methylation
    expr_data = pd.read_csv(expression_path, nrows=5)
    print("\nExpression data preview (first 5 rows):")
    print(expr_data.head())

# Based on the data examination, determine availability and conversion functions

# 1. Gene Expression Data Availability
# If gene expression data exists and appears to be gene expression (not miRNA or methylation)
is_gene_available = gene_file_exists  # We'll assume it's gene expression if the file exists

# 2. Variable Availability and Data Type Conversion

# For Metabolic Rate (trait)
# Let's initialize to None and update based on data examination
trait_row = None
age_row = None
gender_row = None

# Function to convert trait values
def convert_trait(value):
    if pd.isna(value) or value == "nan":
        return None
    
    # Extract the value after the colon if present
    if ":" in str(value):
        value = str(value).split(":", 1)[1].strip()
    
    # Convert to float, handle expected formats
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# Function to convert age values
def convert_age(value):
    if pd.isna(value) or value == "nan":
        return None
    
    # Extract the value after the colon if present
    if ":" in str(value):
        value = str(value).split(":", 1)[1].strip()
    
    # Try to extract age as a number
    try:
        # Look for numbers in the string
        import re
        numbers = re.findall(r'\d+', str(value))
        if numbers:
            return float(numbers[0])
        return None
    except (ValueError, TypeError):
        return None

# Function to convert gender values
def convert_gender(value):
    if pd.isna(value) or value == "nan":
        return None
    
    # Extract the value after the colon if present
    if ":" in str(value):
        value = str(value).split(":", 1)[1].strip().lower()
    else:
        value = str(value).lower()
    
    # Convert to binary (0 for female, 1 for male)
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# Based on clinical_data examination, we need to assign appropriate row indices
# For this example, I'll use placeholder values that will be updated after seeing the data
trait_row = None  # Will be updated based on data inspection
age_row = None    # Will be updated based on data inspection
gender_row = None # Will be updated based on data inspection

# After examining the clinical data output above, we can determine which rows contain our variables of interest
# For now, we're setting them to None, but we'll update based on the inspection of the data

# Assuming trait_row is determined, check if trait data is available
is_trait_available = trait_row is not None

# 3. Save Metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    print("\nSelected clinical features preview:")
    print(preview_df(selected_clinical))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("No trait data available, skipping clinical feature extraction.")


Files in ../../input/GEO/Metabolic_Rate/GSE61225:
['GSE61225_family.soft.gz', 'GSE61225_series_matrix.txt.gz']
Clinical data file not found.
Background information file not found.

Gene expression data exists: False
No trait data available, skipping clinical feature extraction.


### Step 4: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 26023
First 20 gene/probe identifiers:
Index(['ILMN_1343295', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651238', 'ILMN_1651254',
       'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651281', 'ILMN_1651282', 'ILMN_1651285',
       'ILMN_1651286', 'ILMN_1651303', 'ILMN_1651310', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 5: Gene Identifier Review

In [5]:
# These identifiers with the prefix "ILMN_" are Illumina probe IDs, not human gene symbols.
# They need to be mapped to standard human gene symbols for analysis.

requires_gene_mapping = True


### Step 6: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343295', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651230'], 'Array_Address_Id': ['4490161', '2940221', '650349', '2510494', '2940041'], 'ILMN_Gene': ['GAPDH', 'SLC35E2', 'RPS28', 'IPO13', 'TESSP1'], 'ILMN_Synonyms': ['G3PD; GAPD; MGC88685', 'MGC117254; MGC138494; DKFZp686M0869; FLJ34996; FLJ44537; MGC126715; MGC104754; KIAA0447', nan, 'RANBP13; KIAA0724; IMP13; KAP13', nan], 'ILMN_CHR': ['12', '1', '19', '1', nan], 'ILMN_Probe_Chr_Orientation': ['+', '-', '+', '+', nan], 'SEQUENCE': ['CTTCAACAGCGACACCCACTCCTCCACCTTTGACGCTGGGGCTGGCATTG', 'TCACGGCGTACGCCCTCATGGGGAAAATCTCCCCGGTGACTTTCAGGTCC', 'CGCCACACGTAACTGAGATGCTCCTTTAAATAAAGCGTTTGTGTTTCAAG', 'ACAAGAGGCGGGTGAAGGAGATGGTGAAGGAGTTCACACTGCTGTGCCGG', 'GTGATGTCCCACAGTACACCCAGGCCAAACCCCTCCCAGCTGTTGCTGCT'], 'QCStatus_UCSC': ['OK: single match RefSeq', 'OK: single match mRNA', 'OK: single match mRNA', 'OK: single match RefSeq', 'OK: single match mRNA'], 'UCSC_CountMatches': [10.0, 1.0, 1.0, 

### Step 7: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns for gene mapping
# Looking at the previous output:
# - 'ID' column contains Illumina probe IDs (e.g., 'ILMN_1343295')
# - Several columns contain gene symbols, but 'ILMN_Gene' appears to be the primary one with gene symbols
probe_id_column = 'ID'  # Column with probe identifiers
gene_symbol_column = 'ILMN_Gene'  # Column with gene symbols

# 2. Extract mapping between gene identifiers and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, probe_id_column, gene_symbol_column)
print("\nGene mapping preview (first 5 rows):")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("\nGene expression data after mapping:")
print("Number of genes:", len(gene_data))
print("First 10 genes:")
print(gene_data.index[:10])

# 4. Additional step: normalize gene symbols for consistency 
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nGene expression data after normalization:")
print("Number of genes after normalization:", len(gene_data))
print("First 10 normalized genes:")
print(gene_data.index[:10])



Gene mapping preview (first 5 rows):
{'ID': ['ILMN_1343295', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651230'], 'Gene': ['GAPDH', 'SLC35E2', 'RPS28', 'IPO13', 'TESSP1']}

Gene expression data after mapping:
Number of genes: 17047
First 10 genes:
Index(['A1CF', 'A26A1', 'A26B1', 'A26C3', 'A2BP1', 'A2M', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1'],
      dtype='object', name='Gene')



Gene expression data after normalization:
Number of genes after normalization: 16582
First 10 normalized genes:
Index(['A1CF', 'A2M', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS',
       'AADAC', 'AADACL2'],
      dtype='object', name='Gene')


### Step 8: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Create clinical data from the sample characteristics dictionary
# Extract metabolic rate (the actual trait), gender, and age from the clinical data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
characteristics_dict = get_unique_values_by_row(clinical_data)

# Define conversion functions for the clinical features
def convert_metabolic_rate(value):
    if pd.isna(value) or value == "nan":
        return None
    # Extract numeric value after the colon
    if ":" in str(value):
        value = str(value).split(":", 1)[1].strip()
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    if pd.isna(value) or value == "nan":
        return None
    if ":" in str(value):
        value = str(value).split(":", 1)[1].strip().lower()
    else:
        value = str(value).lower()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    return None

def convert_age(value):
    if pd.isna(value) or value == "nan":
        return None
    if ":" in str(value):
        value = str(value).split(":", 1)[1].strip()
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# Create the clinical dataframe with the trait and demographic features
clinical_df = geo_select_clinical_features(
    clinical_data,
    trait="Metabolic_Rate",
    trait_row=4,  # Row 4 contains metabolic equivalents
    convert_trait=convert_metabolic_rate,
    gender_row=5,  # Row 5 contains gender information
    convert_gender=convert_gender,
    age_row=6,  # Row 6 contains age information
    convert_age=convert_age
)

print("Clinical data preview:")
print(preview_df(clinical_df.T))  # Transpose for better viewing

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.T.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_normalized)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, 'Metabolic_Rate')
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Metabolic_Rate')

# 6. Validate the dataset and save cohort information
note = "Dataset contains gene expression data from subjects before and after swimming in a chlorinated pool, with metabolic rate measurements."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Shape of gene data after normalization: (16582, 74)


Saved normalized gene data to ../../output/preprocess/Metabolic_Rate/gene_data/GSE61225.csv
Clinical data preview:
{'Metabolic_Rate': [0.681873816522757, 0.8523423, 0.665104620956877, 0.8313807, 0.786010716020953], 'Age': [31.60849, 31.60849, 24.39425, 24.39425, 51.2115], 'Gender': [0.0, 0.0, 0.0, 0.0, 1.0]}
Saved clinical data to ../../output/preprocess/Metabolic_Rate/clinical_data/GSE61225.csv
Shape of linked data: (74, 16585)


Shape of linked data after handling missing values: (74, 16585)
Quartiles for 'Metabolic_Rate':
  25%: 0.7391046411728177
  50% (Median): 0.883957846599822
  75%: 2.267302
Min: 0.595372813525237
Max: 5.917071
The distribution of the feature 'Metabolic_Rate' in this dataset is fine.

Quartiles for 'Age':
  25%: 27.32101
  50% (Median): 30.16838
  75%: 33.63176
Min: 22.71869
Max: 51.2115
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 30 occurrences. This represents 40.54% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Metabolic_Rate/GSE61225.csv
